엔지니어는 절삭속도(A), 공구기하학(B), 절삭각(C)이 공구수명(시간당)에 미치는 효과에 대해 관심이 있다. 각 요소들은 두 개의 수준을 유지한다.
관측치들에 대해 ABC가 중첩되며, 두 개의 블록에 관측치가 각 4개씩 존재한다.

|A|B|C|처리조합|I|
|---|---|---|---|---|
|-|-|-|(1)|22|
|+|-|-|a|32|
|-|+|-|b|35|
|+|+|-|ab|55|
|-|-|+|c|44|
|+|-|+|ac|40|
|-|+|+|bc|60|
|+|+|+|abc|39|

# (1) 요인효과를 추정하라.

In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.DataFrame({
    'a': [-1, 1, -1, 1, -1, 1, -1, 1],
    'b': [-1, -1, 1, 1, -1, -1, 1, 1],
    'c': [-1,-1,-1,-1, 1, 1, 1, 1],
    '처리조합': ['(1)', 'a', 'b', 'ab', 'c', 'ac', 'bc', 'abc'],
    'Y': [22, 32, 35, 55, 44, 40, 60, 39]
})

df

,a,b,c,처리조합,Y
0,-1,-1,-1,(1),22
1,1,-1,-1,a,32
2,-1,1,-1,b,35
3,1,1,-1,ab,55
4,-1,-1,1,c,44
5,1,-1,1,ac,40
6,-1,1,1,bc,60
7,1,1,1,abc,39


In [12]:
df['ABCBlock'] = df[['a','b','c']].prod(1)
df

,a,b,c,처리조합,Y,ABCBlock
0,-1,-1,-1,(1),22,-1
1,1,-1,-1,a,32,1
2,-1,1,-1,b,35,1
3,1,1,-1,ab,55,-1
4,-1,-1,1,c,44,1
5,1,-1,1,ac,40,-1
6,-1,1,1,bc,60,-1
7,1,1,1,abc,39,1


In [24]:
from DOE import effect_est

for fac in df['처리조합']:
    if fac == '(1)': continue
    if fac == 'abc': 
        print(f'Effect of {fac.upper()}Block:',effect_est(data=df, sign_col=[f for f in fac], value_cols=['Y']))
        break
    print(f'Effect of {fac.upper()}:',effect_est(data=df, sign_col=[f for f in fac], value_cols=['Y']))

Effect of A: 1.25
Effect of B: 12.75
Effect of AB: -1.75
Effect of C: 9.75
Effect of AC: -13.75
Effect of BC: -5.25
Effect of ABCBlock: -6.75


절대값 기준 요인효과 크기 순서:

$$
AC > B > C > ABC > BC > AB > A
$$

# 분산분석

In [16]:
model = sm.OLS.from_formula(formula='Y ~ a + b + c + a:c + ABCBlock', data=df).fit()
model.summary()

c:\Users\USER\miniconda3\envs\doe_env\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     6.450
Date:                Wed, 04 Dec 2024   Prob (F-statistic):              0.140
Time:                        15:35:59   Log-Likelihood:                -19.494
No. Observations:                   8   AIC:                             50.99
Df Residuals:                       2   BIC:                             51.46
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.8750      1.957     20.891      0.002      32.457      49.293
a              0.6250      1.957      0.319      0.780      -7.793       9.043
b              6.3750      1.957      3.258      0.083      -2.043      14.793
c              4.8750      1.957      2.492      0.130      -3.543      13.293
a:c           -6.8750      1.957     -3.514      0.072     -15.293       1.543
ABCBlock      -3.3750      1.957     -1.725      0.227     -11.793       5.043
==============================================================================
Omnibus:                        4.184   Durbin-Watson:                   1.100
Prob(Omnibus):                  0.123   Jarque-Bera (JB):                0.897
Skew:                           0.000   Prob(JB):                        0.639
Kurtosis:                       1.360   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
sm.stats.anova_lm(model, typ=3)

,sum_sq,df,F,PR(>F)
Intercept,13366.125,1.0,436.444898,0.002283
a,3.125,1.0,0.102041,0.779674
b,325.125,1.0,10.616327,0.082680
c,190.125,1.0,6.208163,0.130322
a:c,378.125,1.0,12.346939,0.072316
ABCBlock,91.125,1.0,2.975510,0.226675
Residual,61.250,2.0,NaN,NaN
